## 1. Data import

Import the simulated data and convert in into 3x3x2 ndarray. 3x3 is the tic-tac-toe grid dimensions. 2 are the values of X and O. 1-0 represents X, 0-1 represents O, 0-0 represents empty cell.

Input data representation for the model is:
* Input: current state of 3x3x2 (board)
* Output(Int 1-9) : Position of next move to make

In [177]:
import numpy as np

In [178]:
input_file = 'good_games_data.csv'
raw_data = np.genfromtxt(input_file, delimiter=',')
print("Length of raw data:", len(raw_data))
print("Sample row:", raw_data[0])

Length of raw data: 15119
Sample row: [1. 2. 3. 4. 5. 6. 7. 0. 0. 1.]


In [179]:
def playNextMove(board, move, player):
    playerIndex = player - 1
    moveIndex = move - 1
    
    r = moveIndex // 3
    c = moveIndex % 3
    board[r, c, playerIndex] = 1

In [180]:
def convertRawToModelInput(raw_data):
    X = []
    Y = []
    for raw in raw_data:
        board = np.zeros((3,3,2))
        player = 0
        for i in range(9):
            if raw[i] == 0:
                break #game already over

            if raw[-1] == 1:
                if i%2 == 0:
                    player = 1
                    X.append(board.copy())
                    Y.append(raw[i])
                else:
                    player = 2

            else:           
                if i%2 == 1:
                    player = 2
                    X.append(board.copy())
                    Y.append(raw[i])
                else:
                    player = 1

            playNextMove(board, int(raw[i]), player)

    return np.array(X), np.array(Y)


In [181]:
def shuffle_in_unison(a, b):
     n_elem = a.shape[0]
     indeces = np.random.choice(n_elem, size=n_elem, replace=False)
     return a[indeces], b[indeces]

In [182]:
def convertToOneHot(Y, n_y):
    Y = Y.astype(int)
    Y_index = Y - 1  # convert position to index
    m = len(Y)
    Y_oh = np.zeros((m, n_y))
    Y_oh[np.arange(m), Y_index] = 1
    return Y_oh

In [183]:
X, Y = convertRawToModelInput(raw_data)
X,Y = shuffle_in_unison(X, Y)
Y = convertToOneHot(Y, 9)
print("Size of input data:", len(Y))

Size of input data: 53792


In [184]:
m = 50000
X_train = X[:m]
Y_train = Y[:m]
X_test = X[m:]
Y_test = Y[m:]

print("Shape of X_train", X_train.shape)
print("Shape of Y_train", Y_train.shape)
print("Shape of X_test", X_test.shape)
print("Shape of Y_test", Y_test.shape)

Shape of X_train (50000, 3, 3, 2)
Shape of Y_train (50000, 9)
Shape of X_test (3792, 3, 3, 2)
Shape of Y_test (3792, 9)


## 2. Model

Return instance of the model

In [205]:
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Flatten
from keras.optimizers import Adam

In [211]:
def model():
    
    X_train = Input(shape=(3,3,2,))
    X = Conv2D(128, (3,3), input_shape=(3, 3, 2), activation="relu")(X_train)
    X = Flatten()(X)
    X = Dense(128, activation="relu")(X)
    X = Dense(64, activation="relu")(X)
    X = Dense(32, activation="relu")(X)
    Y = Dense(9, activation="softmax")(X)
    
    model = Model(inputs=X_train, outputs= Y)
    return model

In [212]:
m = model()
opt = Adam()
m.compile(opt, loss="categorical_crossentropy", metrics=['accuracy'])

In [213]:
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 3, 3, 2)           0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 1, 128)         2432      
_________________________________________________________________
flatten_2 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_26 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_27 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_28 (Dense)             (None, 9)                 297       
Total para

In [214]:
m.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
50000/50000 [==============================] - 5s 99us/step - loss: 1.6291 - acc: 0.3595
Epoch 2/10
50000/50000 [==============================] - 4s 87us/step - loss: 1.4526 - acc: 0.4162
Epoch 3/10
50000/50000 [==============================] - 4s 88us/step - loss: 1.4174 - acc: 0.4250
Epoch 4/10
50000/50000 [==============================] - 4s 89us/step - loss: 1.3935 - acc: 0.4277
Epoch 5/10
50000/50000 [==============================] - 4s 87us/step - loss: 1.3752 - acc: 0.4323
Epoch 6/10
50000/50000 [==============================] - 5s 92us/step - loss: 1.3650 - acc: 0.4292
Epoch 7/10
50000/50000 [==============================] - 4s 88us/step - loss: 1.3552 - acc: 0.4302
Epoch 8/10
50000/50000 [==============================] - 4s 89us/step - loss: 1.3510 - acc: 0.4315
Epoch 9/10
50000/50000 [==============================] - 5s 90us/step - loss: 1.3449 - acc: 0.4314
Epoch 10/10
50000/50000 [==============================] - 5s 92us/step - loss: 1.3424 - acc: 0.4316

In [215]:
m.evaluate(X_test, Y_test, batch_size=32)

3792/3792 [==============================] - 0s 59us/step


[1.3555394476476097, 0.4185126582278481]